# Correcting field intensity: calibrating accelerometer and magnetometer

## Load and inspect data
Load pickle file and inspect contents

In [ ]:
import os
import pickle

# Import necessary pyologger utilities
from pyologger.load_data.datareader import DataReader
from pyologger.load_data.metadata import Metadata
from pyologger.plot_data.plotter import *
from pyologger.process_data.sampling import *
from pyologger.calibrate_data.zoc import *
from pyologger.plot_data.plotter import plot_depth_correction
from pyologger.calibrate_data.calibrate_acc_mag import *

# Change the current working directory to the root directory
# os.chdir("/Users/fbar/Documents/GitHub/pyologger")
os.chdir("/Users/jessiekb/Documents/GitHub/pyologger")

root_dir = os.getcwd()
data_dir = os.path.join(root_dir, "data")
color_mapping_path = os.path.join(root_dir, "color_mappings.json")

# Verify the current working directory
print(f"Current working directory: {root_dir}")

In [ ]:
# Initialize the info class
metadata = Metadata()
metadata.fetch_databases(verbose=False)

# Save databases
deployment_db = metadata.get_metadata("deployment_DB")
logger_db = metadata.get_metadata("logger_DB")
recording_db = metadata.get_metadata("recording_DB")
animal_db = metadata.get_metadata("animal_DB")

# Assuming you have the metadata and deployment_db loaded:
datareader = DataReader()
deployment_folder = datareader.check_deployment_folder(deployment_db, data_dir)

In [ ]:
# Load the data_reader object from the pickle file
pkl_path = os.path.join(deployment_folder, 'outputs', 'data.pkl')

with open(pkl_path, 'rb') as file:
    data_pkl = pickle.load(file)

for logger_id, info in data_pkl.logger_info.items():
    sampling_frequency = info.get('datetime_metadata', {}).get('fs', None)
    if sampling_frequency is not None:
        # Format the sampling frequency to 5 significant digits
        print(f"Sampling frequency for {logger_id}: {sampling_frequency} Hz")
    else:
        print(f"No sampling frequency available for {logger_id}")

## Calibrate ACC & MAG

In [ ]:
# Check sampling frequencies of accelerometer and magnetometer
acc_fs = data_pkl.sensor_info['accelerometer']['sampling_frequency']
acc_fs
mag_fs = data_pkl.sensor_info['magnetometer']['sampling_frequency']
mag_fs
print(f"Sampling frequency of Accelerometer is {acc_fs/mag_fs}X of Magnetometer.")

In [ ]:
data_pkl.sensor_data['magnetometer'][['mx', 'my', 'mz']]

In [ ]:
acc_data = data_pkl.sensor_data['accelerometer'][['ax','ay','az']]
mag_data = data_pkl.sensor_data['magnetometer'][['mx', 'my', 'mz']]

upsampled_columns = []
for col in mag_data.columns:
    upsampled_col = upsample(mag_data[col].values, acc_fs / mag_fs, len(acc_data))  # Apply upsample to each column
    upsampled_columns.append(upsampled_col)  # Append the upsampled column to the list

# Combine the upsampled columns back into a NumPy array
mag_data_upsampled = np.column_stack(upsampled_columns)
mag_data = mag_data_upsampled

acc_data = acc_data.values

# Assuming acc_data and mag_data_upsampled are NumPy arrays
print(f"Magnetometer upsampled to match accelerometer length: acc_data shape= {acc_data.shape}, upsampled mag_data shape = {mag_data.shape}")
sampling_rate = acc_fs

In [ ]:
acc_data.shape
mag_data.shape

In [ ]:
# Call the check_AM function
AMcheck = compute_field_intensity_and_inclination(acc_data, mag_data, sampling_rate)

# Access the field intensity and inclination angle
field_intensity_acc = AMcheck['field_intensity'][:, 0]  # Field intensity of accelerometer data
field_intensity_mag = AMcheck['field_intensity'][:, 1]  # Field intensity of magnetometer data
inclination_angle = AMcheck['inclination_angle']

# Print results
print("Field Intensity:\n", field_intensity_acc)
print("Field Intensity:\n", field_intensity_mag)
print("Inclination Angle (degrees):\n", inclination_angle)

## Add interim data to pickle

In [9]:
# Split into calibration_acc, calibration_mag, and inclination_angle

# Calibration for Accelerometer (field_intensity_acc)
calibration_acc = pd.DataFrame({
    'datetime': data_pkl.sensor_data['accelerometer']['datetime'],
    'field_intensity_acc': field_intensity_acc
})
data_pkl.derived_data['calibration_acc'] = calibration_acc
data_pkl.derived_info['calibration_acc'] = {
    "channels": ["field_intensity_acc"],
    "metadata": {
        'field_intensity_acc': {'original_name': 'Field Intensity Acc (m/s^2)',
                                'unit': 'm/s^2',
                                'sensor': 'accelerometer'}
    },
    "derived_from_sensors": ["accelerometer"],
    "transformation_log": ["checked_field_intensity"]
}

# Calibration for Magnetometer (field_intensity_mag)
calibration_mag = pd.DataFrame({
    'datetime': data_pkl.sensor_data['accelerometer']['datetime'],
    'field_intensity_mag': field_intensity_mag
})
data_pkl.derived_data['calibration_mag'] = calibration_mag
data_pkl.derived_info['calibration_mag'] = {
    "channels": ["field_intensity_mag"],
    "metadata": {
        'field_intensity_mag': {'original_name': 'Field Intensity Mag (uT)',
                                'unit': 'uT',
                                'sensor': 'magnetometer'}
    },
    "derived_from_sensors": ["magnetometer"],
    "transformation_log": ["checked_field_intensity"]
}

# Inclination Angle
inclination_angle_df = pd.DataFrame({
    'datetime': data_pkl.sensor_data['accelerometer']['datetime'],
    'inclination_angle': inclination_angle
})
data_pkl.derived_data['inclination_angle'] = inclination_angle_df
data_pkl.derived_info['inclination_angle'] = {
    "channels": ["inclination_angle"],
    "metadata": {
        'inclination_angle': {'original_name': 'Inclination Angle (deg)',
                              'unit': 'deg',
                              'sensor': 'extra'}
    },
    "derived_from_sensors": ["accelerometer", "magnetometer"],
    "transformation_log": ["calculated_inclination_angle"]
}


## Add interim data to pickle

In [ ]:
OVERLAP_START_TIME = '2024-01-16 09:30:00'  # Start time for plotting
OVERLAP_END_TIME = '2024-01-16 10:30:00'  # End time for plotting
ZOOM_START_TIME = '2024-01-16 10:00:00'  # Start time for zooming
ZOOM_END_TIME = '2024-01-16 10:02:30'  # End time for zooming
TARGET_SAMPLING_RATE = 10

notes_to_plot = {
    'heartbeat_manual_ok': {'sensor': 'ecg', 'symbol': 'triangle-down', 'color': 'blue'},
    'heartbeat_auto_detect_accepted': {'sensor': 'ecg', 'symbol': 'triangle-up', 'color': 'green'},
    'heartbeat_auto_detect_rejected': {'sensor': 'ecg', 'symbol': 'triangle-up', 'color': 'red'}
}

fig = plot_tag_data_interactive5(
    data_pkl=data_pkl,
    sensors=['ecg', 'accelerometer', 'magnetometer'],
    derived_data_signals=['depth', 'calibration_acc', 'calibration_mag', 'inclination_angle'],
    channels={},
    time_range=(OVERLAP_START_TIME, OVERLAP_END_TIME),
    note_annotations=notes_to_plot,
    color_mapping_path=color_mapping_path,
    target_sampling_rate=TARGET_SAMPLING_RATE,
    zoom_start_time=ZOOM_START_TIME,
    zoom_end_time=ZOOM_END_TIME,
    zoom_range_selector_channel='depth',
    plot_event_values=[],
)

In [ ]:
acc_data.shape

### Apply adjustments - pick up here

In [ ]:
# Apply the fix_offset_3d function to adjust accelerometer data
result = estimate_offset_triaxial(acc_data)

# Extract the adjusted data and calibration info
adjusted_data_acc = result['X']
calibration_info_acc = result['G']

print("Adjusted Data:\n", adjusted_data_acc)
print("Calibration Info:\n", calibration_info_acc)

In [ ]:
adjusted_data_acc[:,0]

In [34]:
# Calibration for Accelerometer (field_intensity_acc and ax, ay, az)
calibration_acc = pd.DataFrame({
    'datetime': data_pkl.sensor_data['accelerometer']['datetime'],
    'ax': adjusted_data_acc[:,0],
    'ay': adjusted_data_acc[:,1],
    'az': adjusted_data_acc[:,2]
})

data_pkl.derived_data['calibrated_acc'] = calibration_acc
data_pkl.derived_info['calibrated_acc'] = {
    "channels": ["ax", "ay", "az"],
    "metadata": {
        'ax': {'original_name': 'Acceleration X (m/s^2)',
               'unit': 'm/s^2',
               'sensor': 'accelerometer'},
        'ay': {'original_name': 'Acceleration Y (m/s^2)',
               'unit': 'm/s^2',
               'sensor': 'accelerometer'},
        'az': {'original_name': 'Acceleration Z (m/s^2)',
               'unit': 'm/s^2',
               'sensor': 'accelerometer'}
    },
    "derived_from_sensors": ["accelerometer"],
    "transformation_log": ["estimated_offset_triaxial"]
}

In [ ]:
# Apply the fix_offset_3d function to adjust magnetometer data
result = estimate_offset_triaxial(mag_data)

# Extract the adjusted data and calibration info
adjusted_data_mag = result['X']
calibration_info_mag = result['G']

print("Adjusted Data:\n", adjusted_data_mag)
print("Calibration Info:\n", calibration_info_mag)

### Re-run check AM. 
Re-run the field intensity and inclination angle calculations to see if now the values are closer to expected.

In [36]:
# Calibration for Magnetometer (field_intensity_mag and mx, my, mz)
calibration_mag = pd.DataFrame({
    'datetime': data_pkl.sensor_data['accelerometer']['datetime'],
    'mx': adjusted_data_mag[:,0],
    'my': adjusted_data_mag[:,1],
    'mz': adjusted_data_mag[:,2]
})

data_pkl.derived_data['calibrated_mag'] = calibration_mag
data_pkl.derived_info['calibrated_mag'] = {
    "channels": ["mx", "my", "mz"],
    "metadata": {
        'mx': {'original_name': 'Magnetometer (uT)',
               'unit': 'uT',
               'sensor': 'magnetometer'},
        'my': {'original_name': 'Magnetometer (uT)',
               'unit': 'uT',
               'sensor': 'magnetometer'},
        'mz': {'original_name': 'Magnetometer (uT)',
               'unit': 'uT',
               'sensor': 'magnetometer'}
    },
    "derived_from_sensors": ["magnetometer"],
    "transformation_log": ["estimated_offset_triaxial"]
}

In [37]:
# Assuming acc_data and mag_data are extracted from the calibrated accelerometer and magnetometer data
acc_data = data_pkl.derived_data['calibrated_acc'][['ax', 'ay', 'az']].values
mag_data = data_pkl.derived_data['calibrated_mag'][['mx', 'my', 'mz']].values
sampling_rate = 100  # Adjust this to the correct sampling rate of your data

# Call the check_AM function
AMcheck = compute_field_intensity_and_inclination(acc_data, mag_data, sampling_rate)

# Access the field intensity and inclination angle
field_intensity_acc = AMcheck['field_intensity'][:, 0]  # Field intensity of accelerometer data
field_intensity_mag = AMcheck['field_intensity'][:, 1]  # Field intensity of magnetometer data
inclination_angle = AMcheck['inclination_angle']

# Append the new field intensity and inclination angle to calibration_acc and calibration_mag

# Calibration for Accelerometer (append field_intensity_acc)
data_pkl.derived_data['calibration_acc']['calibrated_field_intensity_acc'] = field_intensity_acc

# Calibration for Magnetometer (append field_intensity_mag)
data_pkl.derived_data['calibration_mag']['calibrated_field_intensity_mag'] = field_intensity_mag

# Inclination Angle (append inclination_angle)
data_pkl.derived_data['inclination_angle']['calibrated_inclination_angle'] = inclination_angle

# Update the derived_info to reflect the new columns for accelerometer and magnetometer
data_pkl.derived_info['calibration_acc']["channels"].append("calibrated_field_intensity_acc")
data_pkl.derived_info['calibration_acc']["metadata"]['calibrated_field_intensity_acc'] = {
    'original_name': 'Calibrated Field Intensity Acc (m/s^2)',
    'unit': 'm/s^2',
    'sensor': 'accelerometer'
}

data_pkl.derived_info['calibration_mag']["channels"].append("calibrated_field_intensity_mag")
data_pkl.derived_info['calibration_mag']["metadata"]['calibrated_field_intensity_mag'] = {
    'original_name': 'Calibrated Field Intensity Mag (uT)',
    'unit': 'uT',
    'sensor': 'magnetometer'
}

# Update the derived_info for inclination_angle
data_pkl.derived_info['inclination_angle']["channels"].append("calibrated_inclination_angle")
data_pkl.derived_info['inclination_angle']["metadata"]['calibrated_inclination_angle'] = {
    'original_name': 'Calibrated Inclination Angle (deg)',
    'unit': 'deg',
    'sensor': 'accelerometer, magnetometer'
}


In [ ]:
fig = plot_tag_data_interactive5(
    data_pkl=data_pkl,
    sensors=['ecg', 'accelerometer', 'magnetometer'],
    derived_data_signals=['depth', 'calibrated_acc', 'calibrated_mag', 'inclination_angle', 'calibration_acc', 'calibration_mag'],
    channels={},
    time_range=(OVERLAP_START_TIME, OVERLAP_END_TIME),
    note_annotations=notes_to_plot,
    color_mapping_path=color_mapping_path,
    target_sampling_rate=TARGET_SAMPLING_RATE,
    zoom_start_time=ZOOM_START_TIME,
    zoom_end_time=ZOOM_END_TIME,
    zoom_range_selector_channel='depth',
    plot_event_values=[],
)

### Fix inclination angle
Here is where you would fix the axes of triaxial sensor data (if the sensor axis differs from the tag axis)

In [ ]:
data_pkl.sensor_info['pressure']

### Save data

In [42]:
# Save the new added data to the data structure
with open(pkl_path, 'wb') as file:
        pickle.dump(data_pkl, file)